### Torus Geometry
- **Triangular unit cell**: Primitive vectors G₁ = (G, 0), G₂ = (-G/2, √3G/2)
- **Landau level**: n = 0 (lowest Landau level)
- **Brillouin zone area**: 2π/l² = √0.75 × G²
- **Magnetic length**: l = √(ℏ/eB) sets the length scale

### Filling Factor
- **ν**: Filling factor 1/3 or others (Laughlin state)
- **Nk**: Number of k-points must be multiple of m for commensurability
- **Ne = Nk * ν**: Number of electrons for ν filling

### Interaction Details
- **Form factor**: V(q) = W₀ × 1/|ql| × tanh(|qD|)
- **Screening**: D/l (finite screening length)
- **Landau level projection**: exp(-0.5 × q²l²) form factor
- **Units**: Energy in units of W₀, length in units of magnetic length l

### Key Physics
- **Magnetic translation algebra**: Implements proper commutation relations
- **Periodic boundary conditions**: Torus geometry with modular parameter τ
- **Momentum conservation**: Total momentum K = (K₁, K₂) is conserved
- **Ground state degeneracy**: m-fold degenerate on torus for Laughlin states

In [ ]:
# Define 3×5 k-mesh (Nk=15) for 1/3 filling Laughlin state calculation
k_list = [0 1 2 0 1 2 0 1 2 0 1 2 0 1 2;
          0 0 0 1 1 1 2 2 2 3 3 3 4 4 4]

# System parameters
Nk = 15         # Total number of k-points
Gk = (3, 5)     # Grid dimensions (G1_direction, G2_direction)
Ne = 5          # Ne electrons for this system, Ne=5 for 1/3 filling, Ne=3 for 1/5 filling

In [ ]:
# Define 3×6 k-mesh (Nk=18) for 1/3 filling Laughlin state calculation

#=

k_list = [0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2;
          0 0 0 1 1 1 2 2 2 3 3 3 4 4 4 5 5 5]

# System parameters
Nk = 18         # Total number of k-points
Gk = (3, 6)     # Grid dimensions (G1_direction, G2_direction)
Ne = 6          # Ne electrons for this system, Ne=5 for 1/3 filling, Ne=3 for 1/5 filling

=#

In [ ]:
# Define k-mesh for triangular lattice
# Using 9×9 k-mesh (Nk=27) for accurate Laughlin state calculation
# Note: This setup needs much more time and memory

#=

k_list = [0 3 6 2 5 8 1 4 7 0 3 6 2 5 8 1 4 7 0 3 6 2 5 8 1 4 7;
          0 0 0 1 1 1 2 2 2 3 3 3 4 4 4 5 5 5 6 6 6 7 7 7 8 8 8]

# System parameters
Nk = 27          # Total number of k-points
Gk = (9, 9)      # Grid dimensions (G1_direction, G2_direction)
# Number of electrons for 1/3 filling
Ne = 9          # N electrons for this system

=#

In [ ]:
# Define k-mesh for triangular lattice
# Using 6×5 k-mesh (Nk=30) for accurate Laughlin state calculation
# Note: This setup needs much more time and memory
# Note: for 10/30 filling, 36GB memory is recommended

#=

k_list = [0 1 2 3 4 5 0 1 2 3 4 5 0 1 2 3 4 5 0 1 2 3 4 5 0 1 2 3 4 5;
          0 0 0 0 0 0 1 1 1 1 1 1 2 2 2 2 2 2 3 3 3 3 3 3 4 4 4 4 4 4]

# System parameters
Nk = 30         # Total number of k-points
Gk = (6, 5)      # Grid dimensions (G1_direction, G2_direction)
Ne = 10         # N electrons for this system

=#

In [ ]:
# Import the momentum-conserved exact diagonalization package
using MomentumED

# Physical parameters for the FQH system
Gl = sqrt(2π/sqrt(0.75))  # Magnetic length scale from Brillouin zone area
D_l = 5.0                  # Screening length / magnetic length (D/l = 5)
W0 = 1.0                   # Interaction strength (energy unit)
G12_angle = 2π/3          # Angle between reciprocal lattice vectors (triangular lattice)

# Define the form factor for Coulomb interaction in Landau level
# This is the Fourier transform of the projected Coulomb interaction
# V(q) = W₀ * 1/|ql| * tanh(|qD|) * exp(-0.5 * q²l²)
# The exp(-0.5 * q²l²) factor comes from Landau level projection
function VFF(q1::Float64, q2::Float64)
    ql = sqrt(q1^2 + q2^2 + 2cos(G12_angle) * q1*q2) * Gl  # |q| in magnetic length units
    if ql == 0.0
        return W0 * D_l  # Regularization at q=0 (divergent part)
    end
    return W0 / ql * tanh(ql * D_l) * exp(-0.5 * ql^2)
end

# Sign function for reciprocal lattice vectors
# This implements the phase structure of the magnetic translation group
# The sign depends on the parity of the reciprocal lattice vector indices
function ita(g1::Int64, g2::Int64)
    if iseven(g1) && iseven(g2)
        return 1
    else
        return -1
    end
end

# Cross product for 2D vectors (returns scalar z-component)
# Used for computing geometric phases in the magnetic translation algebra
function ql_cross(q1_1, q1_2, q2_1, q2_2)
    return q1_1 * q2_2 - q1_2 * q2_1
end

# Two-body interaction matrix element
# This implements the full Coulomb interaction with proper magnetic translation phases
# The interaction is computed in momentum space with Landau level projection
# Momentum inputs are Tuple(Float64, Float64) representing (k1, k2) in ratio of Gk
function V_int(kf1, kf2, ki2, ki1, cf1=1, cf2=1, ci2=1, ci1=1)::ComplexF64
    
    # Calculate momentum transfer (modulo reciprocal lattice)
    q = rem.(ki1 .- kf1, 1, RoundNearest)
    G_shift1 = round.(Int64, ki1 .- kf1 .- q, RoundNearest)
    G_shift2 = round.(Int64, kf2 .- ki2 .- q, RoundNearest)

    V_total = ComplexF64(0.0)
    # Sum over reciprocal lattice vectors for convergence
    # Nshell = 2 provides good convergence for this system
    Nshell = 2
    for g1 in -Nshell:Nshell, g2 in -Nshell:Nshell
        if abs(g1-g2) > Nshell
            continue
        end

        # Construct the full momentum transfer including reciprocal lattice
        qq1 = q[1] + g1
        qq2 = q[2] + g2

        # Calculate phase factors from magnetic translation algebra
        # These phases ensure proper commutation relations and gauge invariance
        phase_angle = 0.5ql_cross(ki1[1], ki1[2], kf1[1], kf1[2])
        phase_angle += 0.5ql_cross(ki1[1]+kf1[1], ki1[2]+kf1[2], qq1, qq2)
        phase_angle += 0.5ql_cross(ki2[1], ki2[2], kf2[1], kf2[2])
        phase_angle += 0.5ql_cross(ki2[1]+kf2[1], ki2[2]+kf2[2], -qq1, -qq2)

        phase = cispi(2.0phase_angle)
        sign = ita(g1+G_shift1[1], g2+G_shift1[2]) * ita(g1+G_shift2[1], g2+G_shift2[2])

        V_total += sign * phase * VFF(qq1, qq2)
    end

    return V_total
end

# Create parameter structure for the exact diagonalization
# This contains all the system information needed for the calculation
para = EDPara(k_list = k_list, Gk = Gk, V_int = V_int);

In [ ]:
blocks, block_k1, block_k2, k0number = 
    ED_momentum_block_division(para, ED_mbslist(para, (Ne,)));
length.(blocks)

In [ ]:
scat_list2 = ED_sortedScatteringList_twobody(para);

In [ ]:
Neigen = 5  # Number of eigenvalues to compute per block
energies = Vector{Vector{Float64}}(undef, length(blocks))
vectors = Vector{Vector{Vector{ComplexF64}}}(undef, length(blocks))
for i in eachindex(blocks)
    println("Processing block #$i with size $(length(blocks[i])), momentum $(block_k1[i]), $(block_k2[i])")
    energies[i], vectors[i] = EDsolve(blocks[i], scat_list2; N = Neigen,
        showtime=true, 
    )
end

In [ ]:
# The package for plotting is not included in this package. Use the following to add it:
# using Pkg; Pkg.add("CairoMakie")

In [ ]:
# Plot the energy spectrum
using CairoMakie
CairoMakie.activate!()

begin
    fig = Figure();
    ax = Axis(fig[1, 1])
    # Plot energy levels for each momentum block
    for i in 1:length(blocks)
        for e in energies[i]
            scatter!(ax, i, e, color = :blue, marker=:hline)
        end
    end
    xlims!(-0.2, 1.2+length(blocks))
    fig
end

In [ ]:
# check energies explicitly to see if there's degeneracy in the same momentum sector.
bn = 1 # the block number to inspect
energies[bn]

Compute and plot one-body reduced density matric

In [ ]:
# plot the one-body reduced density matrix of the ground eigenstate in the first block
rdm = ED_onebody_rdm(blocks[bn], vectors[bn][1])
begin
    fig = Figure();
    ax = Axis(fig[1, 1]; yreversed = true)
    hm = heatmap!(ax, abs.(rdm); colorrange = (0,1), 
        colormap = range(Makie.Colors.colorant"white", stop=Makie.Colors.colorant"#ec2f41", length=15)
    )
    Colorbar(fig[1, 2], hm)
    fig
end

Compute the many-body connection and the Wilson loop for many-body Chern number.

In [ ]:
# Define the Landau level infinitesimal form factor
function Landau_ff_inf(k_f, k_i, c=1)
    dk = k_f .- k_i
    k = 0.5 .* (k_f .+ k_i)
    return -π * (k[1]*dk[2] - k[2]*dk[1])
end
para.FF_inf_angle = Landau_ff_inf; # Update the form factor in the parameter

In [ ]:
# boundary twist angle path (Wilson loop)
N_shift = 2  # number of shifts along each edge
shifts = Tuple{Float64, Float64}[(0.0, 0.0)]
for i in 1:N_shift
    push!(shifts, (i/N_shift, 0.0))
end
for i in 1:N_shift
    push!(shifts, (1.0, i/N_shift))
end
for i in 1:N_shift
    push!(shifts, ((N_shift - i)/N_shift, 1.0))
end
for i in 1:N_shift
    push!(shifts, (0.0, (N_shift - i)/N_shift))
end
shifts

If there's only one ground state.

In [ ]:
psi_before = copy(vectors[bn][1])
psi_before *= cis(-angle(psi_before[1]))  # fix global phase

WilsonLoopIntegral= Vector{Float64}(undef, 4N_shift)
for i in eachindex(WilsonLoopIntegral)

    println("k_shift #$i \t $(shifts[i+1])")

    # one-body term is all zero and not changed by k-shift
    scat_list2 = ED_sortedScatteringList_twobody(para; kshift = shifts[i+1]);
    psi_after = EDsolve(blocks[bn], scat_list2; N = 1,
        showtime = true,
    )[2][1]
    psi_after *= cis(-angle(psi_after[1]))  # fix global phase

    WilsonLoopIntegral[i] = ED_connection_step(blocks[bn], 
        psi_after, psi_before, shifts[i+1], shifts[i], para;
        wavefunction_tol = 1e-8, print_amp = true,
        amp_warn_tol = 0.7, amp_warn = true
    )

    psi_before = psi_after
end
ManyBodyChernNumber = sum(WilsonLoopIntegral) / (2π)

If there're 3 ground states in the same total momentum section, we need to use multiple wavefunctions with non-Abelian calculation function.

In [ ]:
psi_before = reduce(hcat, vectors[bn][1:3])
for i in 1:3
    psi_before[:, i] *= cis(-angle(psi_before[1, i]))  # fix global phase
end

using LinearAlgebra
WilsonLoopIntegral= Array{ComplexF64, 3}(undef, 3, 3, 4N_shift)
for i in axes(WilsonLoopIntegral, 3)

    println("k_shift #$i \t $(shifts[i+1])")

    if shifts[i+1] == (0.0, 0.0)
        psi_after = reduce(hcat, vectors[bn][1:3])
        for i in 1:3
            psi_after[:, i] *= cis(-angle(psi_after[1, i]))  # fix global phase
        end
    else
        # one-body term is all zero and not changed by k-shift
        scat_list2 = ED_sortedScatteringList_twobody(para; kshift = shifts[i+1]);
        vecs = EDsolve(blocks[bn], scat_list2; N = 6,
            showtime = false, #vec0 = psi_before[:, 1],
        )[2][1:3]
        psi_after = reduce(hcat, vecs)
        for j in 1:3
            psi_after[:, j] *= cis(-angle(psi_after[1, j]))  # fix global phase
        end
    end

    WilsonLoopIntegral[:, :, i] = ED_NAconnection_step(blocks[bn],
        psi_after, psi_before, shifts[i+1], shifts[i], para;
        wavefunction_tol = 1e-8, print_amp = true,
        amp_warn_tol = 0.7, amp_warn = true
    )

    psi_before = psi_after
end
ManyBodyChernNumber = sum(WilsonLoopIntegral, dims = 3) ./ (2π)
eigvals(ManyBodyChernNumber[:,:,1])

In [ ]:
real.(ManyBodyChernNumber)

In [ ]:
imag.(ManyBodyChernNumber)

In [ ]:
using LinearAlgebra
⊗ = kron
[1; 0; 0] ⊗ vectors[1][1]